In [1]:
import json
import os

import numpy as np
from tqdm import tqdm

In [2]:
dir_path = "data/arc2concept-aug-1000/"

In [3]:
for split in ["train", "test"]:
    print(split)
    indices = np.load(f"{dir_path}/{split}/all__group_indices.npy")
    inputs = np.load(f"{dir_path}/{split}/all__inputs.npy")
    labels = np.load(f"{dir_path}/{split}/all__labels.npy")
    puzzle_identifiers = np.load(f"{dir_path}/{split}/all__puzzle_identifiers.npy")
    puzzle_indices = np.load(f"{dir_path}/{split}/all__puzzle_indices.npy")
    

    num_puzzles = puzzle_indices.shape[0] - 1
    # How many examples per puzzle
    examples_per_puzzle = np.diff(puzzle_indices)  # [N_puzzles]

    # Map each example index -> puzzle index
    puzzle_idx_for_example = np.repeat(
        np.arange(num_puzzles, dtype=np.int32),
        examples_per_puzzle
    )  # [N_examples]

    # Then map puzzle index -> puzzle_identifier
    example_puzzle_identifiers = puzzle_identifiers[puzzle_idx_for_example]  # [N_examples]
    
    puzzlez_with_identifiers = np.concatenate([example_puzzle_identifiers[..., np.newaxis], inputs], axis=-1)
    os.makedirs(f"data/my-arc2concept-aug-1000/{split}", exist_ok=True)
    np.save(f"data/my-arc2concept-aug-1000/{split}/aug_puzzle_idx.npy", example_puzzle_identifiers)
    np.save(f"data/my-arc2concept-aug-1000/{split}/inputs.npy", inputs.astype(np.uint8))
    np.save(f"data/my-arc2concept-aug-1000/{split}/labels.npy", labels.astype(np.uint8))
    
    if split == "train":
        with open(f"data/my-arc2concept-aug-1000/metadata.json", "w") as f:
            json.dump(
                {
                "train": {
                    "num_puzzles": len(indices),
                    "num_examples": "i dunno",
                    "num_aug_puzzles": len(puzzle_identifiers),
                    "num_aug_examples": len(inputs)
                },
            }, f)


train
test


In [4]:
# np.save("data/arc2concept-aug-1000/aug_puzzle_idx.npy", example_puzzle_identifiers)
# np.save("data/arc2concept-aug-1000/inputs.npy", inputs.astype(np.uint8))
# np.save("data/arc2concept-aug-1000/labels.npy", labels.astype(np.uint8))

In [5]:
# with open("data/arc2concept-aug-1000/metadata.json", "w") as f:
#     json.dump(
#         {
#         "train": {
#             "num_puzzles": len(indices),
#             "num_examples": "i dunno",
#             "num_aug_puzzles": len(puzzle_identifiers),
#             "num_aug_examples": len(inputs)
#         },
#     }, f)